<a href="https://colab.research.google.com/github/meashu31/AIML-Capstone-CV9/blob/Vadhi_master/Unet_Models_Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Gogle Drive

In [24]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [13]:
pip install segmentation-models

In [209]:
import os
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import data_utils
import numpy as np
from PIL import Image
from keras.utils.layer_utils import count_params
from tabulate import tabulate
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras.layers import *
from keras.models import *

In [26]:
im_path = '/content/drive/My Drive/Capstone/data/images/'
mask_path = '/content/drive/My Drive/Capstone/data/masks/'
train_list = os.listdir(im_path)
mask_list = os.listdir(mask_path)
print(len(train_list))
print(len(mask_list))

26684
26684


In [219]:
X_train, X_val = train_test_split(train_list[:100], test_size=0.20, random_state=53)
print('Length of training data:',len(X_train),'\nLength of validation data:',len(X_val))

Length of training data: 80 
Length of validation data: 20


In [220]:
class DataGenerator(data_utils.Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, list_IDs, image_path, mask_path, batch_size=32, dim=(1024, 1024),n_channels=1, shuffle=True,resize=False, preprocess_input=preprocess_input):
        """Initialization
        self.list_IDs = List of all unique patient Ids
        self.image_path = path to image folder
        self.mask_path = path to mask folder
        self.batch_size = batch_size
        self.dim = dimenstion for each input image
        self.n_channels = no.of channels per image
        self.resize = if true, will resize the input image to self.dim
        self.shuffle = True to shuffle label indexes after every epoch
        """
        self.list_IDs = list_IDs
        self.TempList = list_IDs
        self.image_path = image_path
        self.mask_path = mask_path
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.resize = resize
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        if len(list_IDs_temp)==0:
          print('Trying to access an empty batch. No image left to iterate over. Try changing the batch index')
          return None
        else:
          # Generate data
          X, y = self.__pixel_generation(list_IDs_temp)

          return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __pixel_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim)
        ''' Initialization
         *self.dim is a variable length parameter. It can vary based on the size of image'''
        
        x_image = np.empty((self.batch_size, self.dim[0],self.dim[1],3))
        x_channels = np.empty((self.dim[0],self.dim[1],3))
        y_mask = np.empty((self.batch_size, *self.dim))
        ID_List = []
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            if self.resize == True:
              x_frame = Image.open(self.image_path + ID).resize((self.dim[0],self.dim[1]),Image.ANTIALIAS)
              y_frame = Image.open(self.mask_path + ID).resize((self.dim[0],self.dim[1]),Image.ANTIALIAS)
            else:
              x_frame = Image.open(self.image_path + ID)
              y_frame = Image.open(self.mask_path + ID)
            #print(np.array(x_frame).dtype)
            # Store images
            for z in range(3):
              x_channels[:,:,z] = np.array(x_frame)
            
            x_image[i,] = preprocess_input(np.array(x_channels, dtype=np.float32))
            # Store masks
            temp = np.array(y_frame)
            temp = np.where(temp > 0 , 1, y_frame)           
            y_mask[i,] = temp
            # Store ID
        return x_image, y_mask

In [71]:
X_train_gen = DataGenerator(list_IDs=X_train , image_path = im_path, mask_path=mask_path,dim=(224,224),batch_size=8,resize=True)
X_Val_gen = DataGenerator(list_IDs=X_val , image_path = im_path, mask_path=mask_path,dim=(224,224),batch_size=8,resize=True)

## Backboned-UNet

In [207]:
def GetModel(Backbone='', pretrained=True,ShowSummery=False):
  """ Loading backbone. """
  # TODO: More backbones
  if Backbone == 'Vanilla_CNN':
    print("Loading Vanilla CNN")
  elif Backbone == 'VGG_16':
    create_model(Backbone='vgg16',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'vgg19':
    create_model(Backbone='vgg19',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'resnet18':
    create_model(Backbone='resnet18',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'resnet34':
    create_model(Backbone='resnet34',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'Resnet-50':
    create_model(Backbone='resnet50',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'resnet101':
    create_model(Backbone='resnet101',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'resnet152':
    create_model(Backbone='resnet152',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'DenseNet121':
    create_model(Backbone='densenet121',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'DenseNet169':
    create_model(Backbone='densenet169',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'DenseNet201':
    create_model(Backbone='densenet201',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'inception_v3':
    create_model(Backbone='inceptionv3',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'Inception_ResNet_V2':
    create_model(Backbone='inceptionresnetv2',pretrained=pretrained,ShowSummery=ShowSummery)
  elif Backbone == 'MobileNet':
    create_model(Backbone='mobilenet',pretrained=pretrained)
  else:
    raise NotImplementedError('{} backbone model is not implemented so far.'.format(name))

In [226]:
def create_model(Backbone='vgg16',pretrained=True,ShowSummery=False):
  # preprocess input
  preprocess_input = get_preprocessing(Backbone)
  x_train = DataGenerator(list_IDs=X_train , image_path = im_path, mask_path=mask_path,dim=(224,224),batch_size=8,resize=True, preprocess_input=preprocess_input)
  x_val = DataGenerator(list_IDs=X_val , image_path = im_path, mask_path=mask_path,dim=(224,224),batch_size=8,resize=True, preprocess_input=preprocess_input)

  # define model
  model = Unet(Backbone, encoder_weights='imagenet', input_shape=(224, 224, 3))
  ##-------------------getLast layer and add reshape----------------------
  LastLayerIdx = len(model.layers) - 1
  LastLayerName = model.layers[LastLayerIdx].name
  for layer in model.layers:
    layer.trainable = not pretrained
    LastLayer = model.get_layer(LastLayerName).output
    x = Reshape((224, 224))(LastLayer)
  model = Model(inputs=model.input, outputs=x)
  trainable_count = count_params(model.trainable_weights)
  non_trainable_count = count_params(model.non_trainable_weights)

  print(tabulate([['Model', 'UNet'],\
                          ['BackBone', Backbone],\
                          ['Total params', model.count_params()],\
                          ['Trainable params', trainable_count],\
                          ['Non-trainable params', non_trainable_count]],\
                          headers=['Header', 'Value'], tablefmt='orgtbl'))
  if ShowSummery: model.summary()

  model.compile('Adam', loss=bce_jaccard_loss, metrics=['accuracy',iou_score])
  checkpoint = ModelCheckpoint("model-{loss:.2f}.h5", monitor="loss", verbose=1, save_best_only=True, save_weights_only=True, mode="min", period=1)
  stop = EarlyStopping(monitor="loss", patience=5, mode="min")

  model.fit(X_train_gen,
          epochs=1,
          steps_per_epoch = len(X_train_gen)/2,
          validation_data=X_Val_gen,
          validation_steps = len(X_train_gen)/3*2,
          callbacks= [checkpoint, stop])

In [228]:
model = GetModel(Backbone='MobileNet',pretrained=True,ShowSummery=False)

| Header               | Value     |
|----------------------+-----------|
| Model                | UNet      |
| BackBone             | mobilenet |
| Total params         | 8336337   |
| Trainable params     | 0         |
| Non-trainable params | 8336337   |
Epoch 1/1
5/5 [==============================] - 26s 5s/step - loss: 2.0372 - accuracy: 0.0048 - iou_score: 4.0810e-11 - val_loss: 3.0995 - val_accuracy: 0.0000e+00 - val_iou_score: 2.9404e-11

Epoch 00001: loss improved from inf to 2.03718, saving model to model-2.04.h5
